# Ground tracks plots

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.graph_objects as go

import ephemerista
from ephemerista.assets import Asset, GroundStation
from ephemerista.plot.groundtrack import GroundTrack
from ephemerista.propagators.sgp4 import SGP4
from ephemerista.time import TimeDelta

In [ ]:
ephemerista.init(eop_path="../tests/resources/finals2000A.all.csv", spk_path="../tests/resources/de440s.bsp")

In [ ]:
# Propgate a trajectory
iss_tle = """ISS (ZARYA)
1 25544U 98067A   24187.33936543 -.00002171  00000+0 -30369-4 0  9995
2 25544  51.6384 225.3932 0010337  32.2603  75.0138 15.49573527461367"""

propagator = SGP4(tle=iss_tle)
start_time = propagator.time
end_time = start_time + TimeDelta.from_hours(6)
times = start_time.trange(end_time, step=float(TimeDelta.from_minutes(1)))
trajectory = propagator.propagate(times)

### Plot 3D trajectory

In [ ]:
fig = go.Figure()

fig.add_trace(trajectory.origin.plot_3d_surface())
fig.add_trace(trajectory.plot_3d())

fig.update_layout(
    title=f"3D trajectory, {trajectory.origin.name}-centered {trajectory.frame.abbreviation} frame",
)
fig.show()

### Plot ground track in 3D

In [ ]:
### Plot Spherical ground track
gt = GroundTrack(trajectory, label="ISS")
gt.plot()

### Plot classical ground track

In [ ]:
# Change projection type
gt.update_projection("equirectangular")
gt.plot()

### Generate ground station network 

In [ ]:
station_data = [
    ("Kiruna", 67.858428, 20.966880),
    ("Esrange Space Center", 67.8833, 21.1833),
    ("Kourou", 5.2360, -52.7686),
    ("Redu", 50.00205516, 5.14518047),
    ("Cebreros", 40.3726, -4.4739),
    ("New Norcia", -30.9855, 116.2041),
]
stations = [Asset(model=GroundStation.from_lla(lon, lat), name=name) for name, lat, lon in station_data]

### Add ground station to ground tracks

In [ ]:
gt.plot_ground_station_network(stations)